<a href="https://colab.research.google.com/github/nirmaljb/deep-learning-notebooks/blob/main/mri_for_alzheimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings

warnings.filterwarnings("ignore")
train_dir = '/content/drive/MyDrive/alzheimer dataset/train'
test_dir = '/content/drive/MyDrive/alzheimer dataset/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.4,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rotation_range=90,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    color_mode='grayscale',
    class_mode='categorical',
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    color_mode='grayscale',
    class_mode='categorical'
)

Found 4647 images belonging to 4 classes.
Found 1813 images belonging to 4 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

CNN1 = Sequential()

CNN1.add(Conv2D(256, (3,3), activation='relu', input_shape=(128,128,1)))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(64, (3,3), activation='relu'))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(256, (3,3), activation='relu'))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(128, (3,3), activation='relu'))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Conv2D(256, (3,3), activation='relu'))
CNN1.add(MaxPooling2D(2,2))

CNN1.add(Flatten())

CNN1.add(Dense(128, activation='relu'))
CNN1.add(Dropout(0.2))

CNN1.add(Dense(64, activation='relu'))

CNN1.add(Dense(4, activation='softmax'))
optimizer = Adam(learning_rate=1e-3)
CNN1.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
CNN1.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 126, 126, 256)  │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 63, 63, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 61, 61, 64)     │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 28, 28, 256)    │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 12, 12, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,027,716 (3.92 MB)

 Trainable params: 1,027,716 (3.92 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
CNN1.fit(train_generator, epochs=50)

Epoch 1/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 29s 162ms/step - accuracy: 0.5548 - loss: 1.0602
Epoch 2/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 148ms/step - accuracy: 0.5882 - loss: 0.9633
Epoch 3/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 150ms/step - accuracy: 0.5914 - loss: 0.9292
Epoch 4/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 21s 146ms/step - accuracy: 0.5879 - loss: 0.9258
Epoch 5/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 151ms/step - accuracy: 0.5978 - loss: 0.8973
Epoch 6/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 147ms/step - accuracy: 0.5946 - loss: 0.8951
Epoch 7/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 148ms/step - accuracy: 0.5991 - loss: 0.8774
Epoch 8/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 150ms/step - accuracy: 0.6013 - loss: 0.8693
Epoch 9/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 21s 144ms/step - accuracy: 0.5850 - loss: 0.8971
Epoch 10/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 150ms/step - accuracy: 0.5952 - loss: 0.8689
Epoch 11/50
 57/146 ━━━━━━━━━━━━━━━━━━━━ 13s 153ms/step - accuracy: 0.5617 - loss: 0.8783